# Day06 Pandas skills: Data Wrangling 
# Pandas操作：資料角力
資料角力(有時也稱為資料改寫)，簡單來說是將資料整理成合乎需求的格式，以利後續視覺化或者套用機器學習模型。今天要來介紹一些運用Pandas操作進行資料角力的方式。

Data wrangling (sometimes referred to as data munging) is the process of transforming and mapping data from a raw data form into another format which is more appropriate for analytics or other purposes like visualization or to fit machine learning models. Today, we are going to talk about some functions in Pandas which are pretty essential for data wrangling.


In [1]:
import pandas as pd

# 連接 .merge()   
用於通過一個或多個鍵將兩個資料集的行連接起來。<br>
Merge two DataFrame through one or more keys.

DataFrame.merge(self, right, how='inner', 
                on=None, left_on=None, right_on=None, left_index=False, right_index=False, 
                sort=False, 
                suffixes=('_x', '_y'), 
                copy=True, indicator=False, validate=None)

## 常用參數說明，小標標註的為該參數預設值：
## Frequently used parameters and their default values:

### how='inner' 
連接的方式，方式有'inner'(內連線，交集)，'left'(左外連線，並集)，'right'(右外連線)，'outer'(全外連線)。

Type of merge to be performed.<br>
'left'/'right': use only keys from left/right frame(SQL left outer join).<br>
'outer': use union of keys from both frames(SQL full outer join).<br>
'inner': use intersection of keys from both frames(SQL inner join).<br>

### on=None, left_on=None, right_on=None, left_index=False, right_index=False
on 用來連接的列索引名稱，在兩個要連接DataFrame物件中都要存在。<br>
left_on, right_on 用左或右側DataFrame連接鍵的列名。<br>
left_index, right_index 用左或右側DataFrame的行索引做為連接鍵。

on：Column or index level names to join on. These must be found in both DataFrames.<br>
left_on, right_on：Column or index level names to join on in the left/right DataFrame.<br>
left_index, right_index：Use the index from the left/right DataFrame as the join key(s). If it is a MultiIndex, the number of keys in the other DataFrame (either the index or a number of columns) must match the number of levels.

如果上述所有連接參數都沒有指定，預設會以兩個DataFrame的列名交集做為連接鍵。<br>
If on is None and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

### sort=True
將合併的資料進行排序。<br>
Sort the joined DataFrame.

### suffixes=('_x', '_y')
要合併的兩個DataFrame如果存在相同列名時，會在列名加上字尾。<br>
Suffix to apply to overlapping column names.

In [2]:
# 創作兩個不同的DataFrame：df1與df2 create two DataFrame df1 and df2
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'], 
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'], 
                    'value': [5, 6, 7, 8]})
print(df1)
print(df2)

  lkey  value
0  foo      1
1  bar      2
2  baz      3
3  foo      5
  rkey  value
0  foo      5
1  bar      6
2  baz      7
3  foo      8


In [3]:
df1.merge(df2, left_on='lkey', right_on='rkey', 
          suffixes=('_left', '_right'))

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,foo,5,foo,5
3,foo,5,foo,8
4,bar,2,bar,6
5,baz,3,baz,7


# 合併 .concat()


沿著指定軸或連接方式將兩個資料框架堆疊(連線、繫結)到一起，相當於資料庫中的全連線。<br>
Concat two DataFrame along axis by stacking(concatenation, binding) them together(SQL union all).

pd.concat(objs, axis=0, 
          join='outer', ignore_index=False, 
          keys=None, levels=None, names=None, 
          verify_integrity=False, sort=None, copy=True)

## 常用參數說明，小標標註的為該參數預設值：
## Frequently used parameters and their default values:

### objs 
要結合的物件，一般而言是列表或字典。<br>
The objects we want to concat, normally list or dictionary.

### axis=0 
指定連線軸向，0為沿索引，1為沿欄位名。<br>
pd.concat([obj1, obj2], axis=0)效果與obj1.append(obj2)相同；<br>
pd.concat([df1, df2], axis=1)效果與df1.merge(df2, left_index=True, right_index=True, how='outer')相同。

The axis to concatenate along.<br>
pd.concat([obj1, obj2], axis=0) is similar to obj1.append(obj2).<br>
pd.concat([df1, df2], axis=1) is similar to df1.merge(df2, left_index=True, right_index=True, how='outer').

### ignore_index=False
改成True重建索引為0, 1,...,n - 1。
If True, use 0, 1,...,n - 1 instead of the original index values along the concatenation axis.

In [4]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
print(s1)
print(s2)

0    a
1    b
dtype: object
0    c
1    d
dtype: object


In [5]:
pd.concat([s1, s2], ignore_index=True)

0    a
1    b
2    c
3    d
dtype: object

In [6]:
pd.concat([s1, s2], keys=['s1', 's2'])

s1  0    a
    1    b
s2  0    c
    1    d
dtype: object

In [7]:
pd.concat([s1, s2], keys=['s1', 's2'], 
          names=['Series name', 'Row ID'])

Series name  Row ID
s1           0         a
             1         b
s2           0         c
             1         d
dtype: object

# 分組 .groupby()
將資料依照需要的欄位名分組。<br>
Group the data by specified column name.

DataFrame.groupby(self, by=None, axis=0, level=None, 
                  as_index=True, sort=True, group_keys=True, 
                  squeeze=False, observed=False, **kwargs)

In [8]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon', 'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
print(df)

   Animal  Max Speed
0  Falcon      380.0
1  Falcon      370.0
2  Parrot       24.0
3  Parrot       26.0


In [9]:
df1 = df.groupby('Animal')
print(df)

   Animal  Max Speed
0  Falcon      380.0
1  Falcon      370.0
2  Parrot       24.0
3  Parrot       26.0


In [10]:
df1.groups

{'Falcon': Int64Index([0, 1], dtype='int64'),
 'Parrot': Int64Index([2, 3], dtype='int64')}

In [11]:
df1.get_group('Falcon') 

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0


In [12]:
df.groupby(['Animal']).mean() # 也可以計算平均 get mean

,Max Speed
Animal,
Falcon,375.0
Parrot,25.0


In [13]:
df.groupby(['Animal']).sum() # 計算總和 get sum

,Max Speed
Animal,
Falcon,750.0
Parrot,50.0


# 移除重複 .drop_duplicates() 
去除特定重複行。<br>
Return DataFrame with duplicate rows removed, optionally only considering certain columns.

DataFrame.drop_duplicates(subset=None, keep='first', inplace=False)

## 常用參數說明，小標標註的為該參數預設值：
## Frequently used parameters and their default values:

### subset=None
用以指定僅考慮特定列有無重複，預設為考慮所有列。<br>
Only consider certain columns for identifying duplicates, by default use all of the columns.

### keep='first'
'first', 'last' 僅保留首/末次出現的項。<br>
False 刪去所有重複。

'first', 'last'：Drop duplicates except for the first/last occurrence.<br>
False：Drop all duplicates.

### inplace=False
刪除重複後要取代原始資料框架或新增一個副本。<br>
Whether to drop duplicates in place or to return a copy.

In [14]:
df = pd.DataFrame({'A':[1, 1, 2, 2], 'B':['a', 'b', 'a', 'b']})
print(df)

   A  B
0  1  a
1  1  b
2  2  a
3  2  b


In [15]:
df.drop_duplicates('B', keep='first', inplace=True)
df

,A,B
0,1,a
1,1,b


許多運算符號(>, ==, <, ~)都可以在Pandas作為篩選條件使用。<br>
Many Conditional operators (>, ==, <, ~) could use in Pandas in conditional screening.

本篇程式碼請參考Github。The code is available on Github.

文中若有錯誤還望不吝指正，感激不盡。<br>
Please let me know if there’s any mistake in this article. Thanks for reading.

Reference 參考資料：

[1] 第二屆機器學習百日馬拉松內容

[2] [Python Data Analysis Library](https://pandas.pydata.org/)

[3] [Data Wrangling with pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

[4] [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#constructor)

[5] [Data wrangling](https://en.wikipedia.org/wiki/Data_wrangling)

[6] [【pandas】[3] DataFrame 資料合併，連線](https://www.itread01.com/content/1549097109.html)

[7] [Pandas之drop_duplicates：去除重复项](https://blog.csdn.net/u010665216/article/details/78559091)
